![alt text][top-banner]

[top-banner]: ./callysto-top-banner.jpg

In [1]:
from IPython.display import HTML
hide_me = ''
HTML('''<script>
code_show=true; 
function code_toggle() {
  if (code_show) {
    $('div.input').each(function(id) {
      el = $(this).find('.cm-variable:first');
      if (id == 0 || el.text() == 'hide_me') {
        $(this).hide();
      }
    });
    $('div.output_prompt').css('opacity', 0);
  } else {
    $('div.input').each(function(id) {
      $(this).show();
    });
    $('div.output_prompt').css('opacity', 1);
  }
  code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code,
<form action="javascript:code_toggle()"><input style="opacity:1" type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
hide_me
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets, Button, Layout
from scipy import stats
from collections import Counter
from array import array
from statistics import mode
import IPython
import pandas
import sys
from astropy.table import Table
import tkinter
import csv
import pandas as pd
from pandas import DataFrame

# Effect of Outliers on Central Tendency
This notebook(CC-66) will explore the effect of outliers on central tendency (mean, median, and mode).

This notebook is organized as follows. 
* Section 1 contains preliminaries, the definition of outliers, and two primary examples. 
* In Section 2, we first compute the basic central tendency without outliers by importing a data set from local folder.
* In Section 3, you'll get to use some basic Python tools to adjust a dataset by adding one or more outliers.<br>
You'll then be able to observe the effect of those outliers on mean, median, and mode. 
* In Section 4, we show how to use Python's statistics libraries to analyze date. 
* Section 5 contains interactive exercises for students. 


##  <font color='Blue'>  Section 1 </font> 
## Preliminaries
The definitions and basic examples for mean, median, and mode were discussed in [a previous notebook](CC65-Central_Tendency.ipynb). <br>
You may want to review that notebook before proceeding.



### 1.1 Outlier : Out, liar
A value that "lies outside" (is much smaller or larger than) most of the other values in a set of data.

##### 1.1.1 Example 
Data set: $2, 26, 23, 27, 25, 28, 29, 24, 99 $
<br> $2$ and $99$ are smaller and larger, respectively, than the other data.
<br> Therefore, the outliers of the data set are $2$ and $99$


##### 1.1.2 Example 
In a math test, ten students obtained following marks (out of $100$): <br>
$78, 87, 85, 96, 84, 92, 102, 79, 81, 97$

In the data set there is no data that is obviously smaller or larger than the other values.<br>
But we are told that the exam was out of $100$, so an individual can get at best $100$ for their mark on the test.<br>
Any mark that exceeds this maximum range should be suspicious. (Maybe there were bonus marks?)<br>
Since it's an unexpected value, $102$ is the outlier in the data set.  

##  <font color='Blue'>  Section 1 </font> 
## Central Tendency Computation: Sodium content 

Now we will calculate the mean, median, and mode for a dataset stored in a local text (csv) file. This particular data happens to be the sodium content per serving in a selection of supermarket items. You'll notice one number in the list below that's far greater than the others: this is the outlier in our dataset. (The product? In case you were wondering, it's soy sauce.)

### 2.1 Import Data
The block of code below imports our data. First we display the full view of the data set.There are 29 products that contain different sodium content as follows.  

In [3]:
df = pd.read_csv('CC66_materials/exampleDataSource.csv')
print(df)

                             Product   Sodium content
0             Tomato sauce (Brand A)              871
1             Tomato sauce (Brand B)             1250
2                          Soy sauce             6458
3                    Tinned tomatoes              270
4                      Four bean mix              250
5                       Corn kernels              205
6                 Mushrooms (tinned)              340
7                       Tomato paste              843
8                 Chunky pasta sauce              482
9                      Chicken stock              521
10                        Beef stock              450
11                           Peanuts              360
12               Beef strogonoff mix              780
13            Dry roasted macadamias              340
14                           Cashews              510
15                            Butter              460
16                  Sunflower spread              380
17                       Bak

Now we will read the data one by one and store them into an array. The first line imports the file. The next lines tell Jupyter how to read the file, and the last line tells Jupyter to display the information it read from the file.

In [4]:
data = []
with open('CC66_materials/exampleData.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        data.append(int(row[1]))
print("Sodium content read from the file: ", data)

Sodium content read from the file:  [871, 1250, 270, 250, 205, 340, 843, 482, 521, 450, 360, 780, 340, 510, 460, 380, 290, 458, 335, 355, 590, 600, 595, 547, 1036, 410, 190, 530]


Now that we've loaded our data, we need to process it. <br>


In [5]:
hide_me
def computeCenTendency(dataset):
    
    #mean value
    mean= np.mean(dataset)
    print("Mean: ", round(mean,3))

    #median value
    # First we need to sort the data in ascending order
    dataset.sort()
    median = np.median(dataset)
    print("Median: ", round(median,3))
    
    # Mode
    hits = []
    for item in dataset:
        tally = dataset.count(item)
        #Makes a tuple that is the number of huts paired with the relevant number
        values = (tally, item)
        # Only add one entry for each number in the set
        if values not in hits:
            hits.append(values)
    hits.sort(reverse=True)
    if hits[0][0]>hits[1][0]:
        print("Mode:", round(hits[0][1],3), "(appeared", hits[0][0], "times.)")
    else:
        print("There is no mode")

    Counter(dataset)
    return mean, median, hits[0][1]


### 2.3 Calculation 
Now we call a function which will return us the mean, median, and mode of a dataset.. The required parameter of the function is the list of data. 

In [6]:
centralTendency = []
centralTendency = computeCenTendency(data)

Mean:  508.857
Median:  459.0
Mode: 340 (appeared 2 times.)


### 2.4 Histogram

Next, we will produce a **histogram** of our data. A histogram is a useful plot for visualizing how the values in our dataset are distributed.<br>
Again, we begin by importing the Python libraries needed to accomplish the task.<br>

Now we can define our histogram plot. The definition depends on a number of parameters, such the number of 'bins'.<br>
When there are a lot of data points, we don't plot each point individually. Instead, we group together nearby values.<br>
Each such grouping, given by a range of values, is called a *bin*.<br>
The height of each bar in the histogram is given by the number of data points in the corresponding bin.

In [7]:
hide_me

def plotHistogram(x_values, num_bins, xLabel, yLabel, histTitle):
    n, bins, patches = plt.hist(x_values, num_bins, facecolor='blue', alpha=0.5)

    plt.xlabel(xLabel)
    plt.ylabel(yLabel)
    plt.title(histTitle)
    plt.show()

Now we define another function to get the value of *bin* value size from the user and generate a histogram based on the value.

In [8]:
hide_me

def callPlottingFunction(num_bins):
    print("Generating... plot for :", num_bins)
    plotHistogram(data, num_bins , 'Sodium Content', 'values', 'Histogram of 30 products in Australian supermarkets')



#### 2.4.1 Interactive histogram
To present the histogram in a interactive way, here we have used a Python widget called a slider.<br>
We invoke a built in function named *interact()*, that automatically creates user interface controls for exploring code and data interactively. 

In the following slider, we can adjust the *bin* size (labeled as *num_bins* ) and generate histogram with the value of num_bins.<br>
Students can change the value between the range of $0$ to $100$. By default the value of *num_bins* is set to 50.<br> 
Actually there is no good bin size. It depends on data set. It has impact on data representation. If you take lower bin size then the data will be congested and scatter for higher bin value. That is why we make $50$ as a default value but the user is requested to move the slider value to the left (say 05) and right (say 95) marginal value and observe the histogram accordingly. In general, if an outlier is more higher than the other data then if you increase the *num_bins* value you will find the outlier, and vice versa. 
To generate the histogram for different values click anywhere on the slider. 

N.B. The interactivity here is a little rough: the graph disappears and reappears each time the slider value changes!

In [9]:
interact(callPlottingFunction, num_bins=widgets.IntSlider(min=0,max=100,step=1,value=50));

interactive(children=(IntSlider(value=50, description='num_bins'), Output()), _dom_classes=('widget-interact',))

##  <font color='Blue'>  Section 2 </font> 
## Exploring the effect of outliers 

In the example above, the sodium content in the soy sauce produced a clear outlier.<br>
Next, we will begin with a dataset that does not initially contain any outliers, and compute its mean, median, and mode.<br>
You will then have the opportunity to add additional data points to the set, and then compute their effect on these statistics.

### 3.1 Dataset without outlier
We remove the soy sauce entry from our list of foods and their sodium content.<br>
Now we perform another iteration to compute every task that we defined in section 2.  

In [10]:
hide_me
dataWithoutOutlier = []
with open('CC66_materials/exampleData_NoOutlier.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        dataWithoutOutlier.append(int(row[1]))
print("Sodium content read from the file: ", dataWithoutOutlier)

centralTendencyWithoutOutlier = []
centralTendencyWithoutOutlier = computeCenTendency(dataWithoutOutlier)

def callPlottingFunctionNoOutlier(num_bins):
    print("Generating... plot for :", num_bins)
    plotHistogram(dataWithoutOutlier, num_bins , 'Sodium Content', 'values', 'Histogram of 30 products in Australian supermarkets')

interact(callPlottingFunctionNoOutlier, num_bins=widgets.IntSlider(min=0,max=100,step=1,value=50));

Sodium content read from the file:  [871, 1250, 270, 250, 205, 340, 843, 482, 521, 450, 360, 780, 340, 510, 460, 380, 290, 458, 335, 355, 590, 600, 595, 547, 1036, 410, 190, 530]
Mean:  508.857
Median:  459.0
Mode: 340 (appeared 2 times.)


interactive(children=(IntSlider(value=50, description='num_bins'), Output()), _dom_classes=('widget-interact',))

### 3.2 Adding an outlier
Here, we provide an opportunity so that student can add outlier by their own choice.<br>
First, they have to enter how many outlier(s) they want to add.<br>
Then the program will take the input outlier(s).  

In [11]:
hide_me
inputs = []
numbers = int(input("Number(s) of Outlier : "))
inputs = [input("Outlier " + str(i+1) + ": ")  for i in range(numbers)]
inputs = list(map(int, inputs))

datasetWithOutlier = []
datasetWithOutlier = dataWithoutOutlier + inputs

if(numbers == 1):
    print("Dataset with outlier (last", numbers, "entry was added) : ", datasetWithOutlier)
else:
    print("Dataset with outliers (last", numbers, "entries were added) : ", datasetWithOutlier)

Number(s) of Outlier : 1
Outlier 1: 23232
Dataset with outlier (last 1 entry was added) :  [190, 205, 250, 270, 290, 335, 340, 340, 355, 360, 380, 410, 450, 458, 460, 482, 510, 521, 530, 547, 590, 595, 600, 780, 843, 871, 1036, 1250, 23232]


Now we will compute the central tendency and produce histogram of the dataset with outlier.

In [12]:
centralTendencyWithOutlier = []
centralTendencyWithOutlier = computeCenTendency(datasetWithOutlier)

Mean:  1292.414
Median:  460.0
Mode: 340 (appeared 2 times.)


### 3.3 Historgram with Outliers
Here, we plot a histogram of the data that contain outlier(s). Student can also define the size of *bins* on the slider and observe the change of grpah representation. 

In [13]:
hide_me
def callPlottingFunctionWithOutlier(num_bins):
    print("Generating... plot for :", num_bins)
    plotHistogram(datasetWithOutlier, num_bins, 'Sodium Content', 'values', 'Histogram with Outliers')
interact(callPlottingFunctionWithOutlier, num_bins=widgets.IntSlider(min=0,max=100,step=1,value=50));

interactive(children=(IntSlider(value=50, description='num_bins'), Output()), _dom_classes=('widget-interact',))

**Effects of Outlier :** From the above results and histogram, we see that adding outliers can change the mean dramatically.<br>
This is because we need to add all the values to determine the mean value.<br>
Outliers are often values that are much larger or smaller than the other values.<br>
When we add these values to the sum, the average can change significantly.

On the contrary, to find out the median value we need to considere the middle number(s).<br>
Adding an outlier adds a data point at the far end of our data set, and everything else shifts over only one spot.

The mode is very unlikely to update, because we define mode by the repetition of values.<br>
Input a very large number as an outlier and observe the histogram. <br>
You will notice all regular data should be in one particular area and the inputted value is far away. Consider the following table to get the clear idea of the effect of outliers on mean, median and mode.

In [14]:
hide_me
centralTendencyWithoutOutlierArray = np.around(np.asarray(centralTendencyWithoutOutlier), 3)
centralTendencyWithOutlierArray = np.around(np.asarray(centralTendencyWithOutlier), 3) 

arr = { 'Central Tendency ':  np.array(['Mean','Median','Mode' ]),
        'Before adding outlier ':  np.array([centralTendencyWithoutOutlierArray[0],  centralTendencyWithoutOutlierArray[1], 
                                            centralTendencyWithoutOutlierArray[2] ]),
        'After adding outlier ': np.array([centralTendencyWithOutlierArray[0], centralTendencyWithOutlierArray[1],
                                           centralTendencyWithOutlierArray[2]])}
print(Table(arr))

Central Tendency  Before adding outlier  After adding outlier 
----------------- ---------------------- ---------------------
             Mean                508.857              1292.414
           Median                  459.0                 460.0
             Mode                  340.0                 340.0


##  <font color='Blue'>  Section 4 </font> 
## Example Using Python Library 
In this section we discuss about pandas dataframe. We use **nba_2013.csv** file as dataset that contains $30$ different criteria of $480$ nba players of 2013. We compute the central tendency of this dataframe on [a previous notebook](CC65-Central_Tendency.ipynb). Now we show the outliers of this dataset. We find out the outliers for **stl** categories. Students can change the category name(e.g. **age, mp, fg, drb, asst, gs, tov...**) on the fourth line (only replace the "stl" with category name) of the following code and observe whether there is any outlier or not. 

In [15]:
nba = pandas.read_csv("CC66_materials/nba_2013.csv")
def callPlottingFunctionNBA(num_bins):
    print("Generating... plot for :", num_bins)
    plotHistogram(nba[["stl"]], num_bins, "nba players", "values", "Histogram of 2013 nba players")
interact(callPlottingFunctionNBA, num_bins = widgets.IntSlider(min=0,max=100,step=1,value=10));

interactive(children=(IntSlider(value=10, description='num_bins'), Output()), _dom_classes=('widget-interact',))

## <font color='Blue'>  Section 5 </font> 
## Conclusion

The three most common statistical averages are (arithmetic) mean, median and mode. We observed the effect of outliers on mean, median,
and mode. Mean updated very quickly for adding outlier(s),median changes very slowly and most cases mode would remain same. 

##  <font color='Blue'>  Section 6 </font> 
## Test yourself 

### 6.1 Practice Problem : Easy
Consider the following GPAs of	students	from two	semesters	of	Stat-0000 course: 
<br>
Semester $1: 3.1, 3.2, 2.8,  2.9, 3.0, 3.4,	2.3, 3.2, 2.1,	3.5$ <br>
Semester $2: 2.2, 4.1,	2.6, 2.7, 3.8, 2.8,	2.4, 3.2, 2.7,	2.9$


In [16]:
hide_me
def display(question, answerList):
    print(question)
    IPython.display.display(answerList)

In [17]:
hide_me
question611 = "6.1.1 What will be the mean of semester 1?"
answer611 = widgets.RadioButtons(options=['Select best one','2.95', '3', '3.05', '3.10', 'None of the above'],
                             value= 'Select best one', description='Choices:')


def checkAnswer(a):
    IPython.display.clear_output(wait=False)
    display(question611, answer611)
    if answer611.value == '3.05':
        print("Correct Answer!")
    else:
        if answer611.value == 'Select best one':
            pass
        else:
            print("Wrong answer! Try again.")

display(question611, answer611)

answer611.observe(checkAnswer, 'value')

6.1.1 What will be the mean of semester 1?


RadioButtons(description='Choices:', options=('Select best one', '2.95', '3', '3.05', '3.10', 'None of the above'), value='Select best one')

In [18]:
%%HTML
<html>
<head>
<script type="text/javascript">

<!--
function toggle(id) {
var e = document.getElementById(id);
if(e.style.display == 'none')
e.style.display = 'block';
else
e.style.display = 'none';
}
//-->
</script>
</head>

<body>
<div id="question611">
 <button id = "611"
onclick="toggle('answer611');">Solution</button> 
</div>
<div style="display:none" id="answer611">
To find out the mean of semester 1 we divide the sum of all data by the numbers of data.<br/>
Therefore, Mean $=  \frac{3.1 + 3.2 + 2.8 + 2.9 + 3.0 + 3.4 + 2.3 + 3.2 + 2.1 + 3.5}{10} = 3.05$ <br/>


</div>

</body>
</html>

In [19]:
hide_me
question612 = "6.1.2 What will be the median of semester 2?"

answer612 = widgets.RadioButtons(options=['Select best one', '2.43', '2.5', '2.67', '2.75', 'None of the above'],
                              value = 'Select best one', description='Choices:')

def check612(b):
    IPython.display.clear_output(wait=False)
    display(question612, answer612)
    if answer612.value == '2.75':
        print("Correct Answer!")
    else:
        if answer612.value == 'Select best one':
            pass
        else:
            print("Wrong answer! Try again.")

IPython.display.clear_output(wait=False)
display(question612, answer612)
answer612.observe(check612, 'value')

6.1.2 What will be the median of semester 2?


RadioButtons(description='Choices:', options=('Select best one', '2.43', '2.5', '2.67', '2.75', 'None of the above'), value='Select best one')

In [20]:
%%html
<html>
<head>
<script type="text/javascript">
<!--
function toggle(id) {
var e = document.getElementById(id);
if(e.style.display == 'none')
e.style.display = 'block';
else
e.style.display = 'none';
}
//-->
</script>
</head>

<body>
<div id="question612">
 <button id = "612"
onclick="toggle('answer612');">Solution</button> 
</div>
<div style="display:none" id="answer612">
To determine the median at first we need to sort the data. <br/>
Sorted data,<br/>
Semester  $2: 2.2,2.4,2.6,2.7,2.7,2.8,2.9,3.2,3.8,4.1$ <br/>
There are $10$ numbers in the data set, so the average of $5^{th}$ and $6^{th}$ will be the median. <br/>
So, the Median $= \frac{2.7 + 2.8}{2} = 2.75$ <br/>


</div>

</body>
</html>

### 6.2 Practice Problem : Medium


#### 6.2.1 Distribution type
We want to know how the values are distributed. To determine the type, if $mean<median<mode$ then it is called negatively skewed and if $mean > median >mode$ then it is positively skewed. 

In [21]:
hide_me
question621 = "6.2.1 What is the distribution type of semester 1? (Hint:first, calcualte mean, median, mode)"

answer621 = widgets.RadioButtons(options=['Select best one', 'Negatively Skewed', 'Positively skewed', 'both of them', 'None of the above'],
                              value = 'Select best one', description='Choices:')

def check621(c):
    IPython.display.clear_output(wait=False)
    display(question621, answer621)
    if answer621.value == 'Negatively Skewed':
        print("Correct Answer!")
    else:
        if answer621.value == 'Select best one':
            pass
        else:
            print("Wrong answer! Try again.")
        
IPython.display.clear_output(wait=False)
display(question621, answer621)
answer621.observe(check621, 'value')

6.2.1 What is the distribution type of semester 1? (Hint:first, calcualte mean, median, mode)


RadioButtons(description='Choices:', options=('Select best one', 'Negatively Skewed', 'Positively skewed', 'both of them', 'None of the above'), value='Select best one')

In [22]:
%%html
<html>
<head>
<script type="text/javascript">
<!--
function toggle(id) {
var e = document.getElementById(id);
if(e.style.display == 'none')
e.style.display = 'block';
else
e.style.display = 'none';
}
//-->
</script>
</head>

<body>
<div id="question621">
 <button id = "621"
onclick="toggle('answer621');">Solution</button> 
</div>
<div style="display:none" id="answer621">
At first we have to calculate the mean, median and mode of semester 1 <br/> 
Mean $=  \frac{3.1 + 3.2 + 2.8 + 2.9 + 3.0 + 3.4 + 2.3 + 3.2 + 2.1 + 3.5}{10} = 2.95$ <br/>
For finding the median we need to sort the data. <br/>
Sorted data,<br/>
Semester  $1: 2.1,2.3,2.8,2.9,3.0,3.1,3.2,3.2,3.4,3.5$ <br/>
There are $10$ numbers in the data set, so the average of $5^{th}$ and $6^{th}$ will be the median. <br/>
Median $= \frac{3.0 + 3.1}{2} = 3.05$ <br/>
From the above sorted data we can see that only $3.2$ repeats twice. <br/>
So, the mode is $3.2$ <br/>

The mean is less than median, and median is less than mode. That means, mean$<$ median $<$ mode. <br/>
Therefore the distribution type of semester 1 data is Negatively Skewed. <br/>

</div>

</body>
</html>

#### 6.2.2 Median of medians
Now, we learn how to calculate the median of medians. We define the term as quartile. When we compute the median (M) of a dataset we divide them into two groups. Let say, in group $1$ all values should be less than or equal of median.
<br>
**First quartile $(M_1):$** the median of first group. Now, we have smaller data sets and we can find the median as we discussed in preliminaries.   <br>
**Third quartile $(M_3)$** the median of second group where every value is equal or greater than median. <br>
The **Second quartile** is our median (M).
<br>
For example, our data set is: 
$S_1 = {2, 5, 7, 8, 9}$<br>
So the median (M) is $: 7$<br>
and now we have two groups:
<br>
Group $1: {2,5}$ <br>
Group $2: {8,9}$ <br>
So the first quartile $(M_1)$ is: $\frac{2+5}{2} = 3.5$ <br>
similarly, third quartile, $(M_3) = 8.5$

<br>
    Based on the discussion, answer the following questions. 

In [23]:
hide_me
answer622 = widgets.RadioButtons(options=[  'Select best one','2.5 and 3.1', '2.6 and 3.1', '2.6 and 3.2', '2.8 and 3.2', 'None of the above'],
                             value = 'Select best one', description='Choices:')

question622 = "6.2.2 What is the first and third quartiles of semester 2?"


def check622(d):
    IPython.display.clear_output(wait=False)
    display(question622, answer622)
    if answer622.value == '2.6 and 3.2':
        print("Correct Answer!")
    else:
        if answer622.value == 'Select best one':
            pass
        else:
            print("Wrong answer! Try again.")
        
IPython.display.clear_output(wait=False)
display(question622,answer622 )
answer622.observe(check622, 'value')

6.2.2 What is the first and third quartiles of semester 2?


RadioButtons(description='Choices:', options=('Select best one', '2.5 and 3.1', '2.6 and 3.1', '2.6 and 3.2', '2.8 and 3.2', 'None of the above'), value='Select best one')

In [24]:
%%html
<html>
<head>
<script type="text/javascript">
<!--
function toggle(id) {
var e = document.getElementById(id);
if(e.style.display == 'none')
e.style.display = 'block';
else
e.style.display = 'none';
}
//-->
</script>
</head>

<body>
<div id="question622">
 <button id = "622"
onclick="toggle('answer622');">Solution</button> 
</div>
<div style="display:none" id="answer622">
At first we have to sort the data of semester 2 <br/> 
Sorted data,
Semester  2: 2.2,2.4,2.6,2.7,2.7,2.8,2.9,3.2,3.8,4.1
Data for first Median :  2.2, 2.4, 2.6, 2.7, 2.7 <br/>
    So the first median or first quartile is 2.6 <br/>
Similarly, third quartile $M_3$ is 3.2 <br/>

</div>

</body>
</html>

### 6.3 Practice Problem : Hard
A student has gotten the following marks on his tests: 87, 95, 76, and 88. He wants an 85 or better overall.

In [25]:
hide_me
answer631 = widgets.RadioButtons(options=[ 'Select best one','78', '78.5', '79', '80', 'None of the above'],
                             value = 'Select best one', description='Choices:')

question631 = "6.3.1 What is the minimum mark he must get on the last test in order to achieve that average?"

def check631(e):
    IPython.display.clear_output(wait=False)
    display(question631, answer631)
    if answer631.value == '79':
        print("Correct Answer!")
    else:
        if answer631.value == 'Select best one':
            pass
        else:
            print("Wrong answer! Try again.")
        
IPython.display.clear_output(wait=False)
display(question631, answer631)
answer631.observe(check631, 'value')

6.3.1 What is the minimum mark he must get on the last test in order to achieve that average?


RadioButtons(description='Choices:', options=('Select best one', '78', '78.5', '79', '80', 'None of the above'), value='Select best one')

In [26]:
%%html
<html>
<head>
<script type="text/javascript">
<!--
function toggle(id) {
var e = document.getElementById(id);
if(e.style.display == 'none')
e.style.display = 'block';
else
e.style.display = 'none';
}
//-->
</script>
</head>

<body>
<div id="question631">
 <button id = "631"
onclick="toggle('answer631');">Solution</button> 
</div>
<div style="display:none" id="answer631">
The minimum mark is what we need to find. To find the average of all his marks (the known ones, plus the unknown one), 
we have to add up all the grades, and then divide by the number of marks. Since we do not have a score for the last test yet, 
we will use a variable to stand for this unknown value: "x". Then computation to find the desired average is: <br />

the first step : (87 + 95 + 76 + 88 + x) ÷ 5 = 85 <br />
Multiplying through by 5 and simplifying, we get: <br/>
the next step : 87 + 95 + 76 + 88 + x = 425 <br />
the next step : 346 + x = 425 <br />
the final step : x = 79 <br />
so, he needs to get at least a 79 on the last test.
</div>

</body>
</html>

### 6.4 Practice Problem: Miscellaneous
In this section we discuss how to determine the outliers based on first and third quartile. First, we need to find out the inter quartile range (IQR). And then we compute the lower and upper limit for outliers by the following formulae: <br>
Lower limit: $$M_1 - 1.5 \times \text{IQR}$$
and upper limit: $$M_3 + 1.5 \times \text{IQR}$$
where,<br>
IQR $= (M_3 - M_1)$ <br>
So, all the data that are not belong to this range are outliers.

Now, answer the following question:

In [27]:
hide_me
answer641 = widgets.RadioButtons(options=['Select best one','1.6 and 4.1', '1.7 and 3.9', '2.2 and 3.8', '1.7 and 4.1', 'None of the above'],
                              value='Select best one',description='Choices:')

question641 = "6.4.1 What is the lower and upper limit of outliers of semester 2?"


def checkAnswer641(f):
    IPython.display.clear_output(wait=False)
    display(question641, answer641 )
    if answer641.value == '1.7 and 4.1':
        print("Correct Answer!")
    else:
        if answer641.value == 'Select best one':
            pass
        else:
            print("Wrong answer! Try again.")

IPython.display.clear_output(wait=False)
display(question641, answer641)
answer641.observe(checkAnswer641, 'value')

6.4.1 What is the lower and upper limit of outliers of semester 2?


RadioButtons(description='Choices:', options=('Select best one', '1.6 and 4.1', '1.7 and 3.9', '2.2 and 3.8', '1.7 and 4.1', 'None of the above'), value='Select best one')

In [28]:
%%html
<html>
<head>
<script type="text/javascript">
<!--
function toggle(id) {
var e = document.getElementById(id);
if(e.style.display == 'none')
e.style.display = 'block';
else
e.style.display = 'none';
}
//-->
</script>
</head>

<body>
<div id="question641">
 <button id = "641"
onclick="toggle('answer641');">Solution</button> 
</div>
<div style="display:none" id="answer641">
At first we have to determine the first median$(M_1)$ and third median $(M_3)$ of semester 2.<br/> 
Data for first Median :  2.2, 2.4, 2.6, 2.7, 2.7 <br/>
    So the first median or first quartile is 2.6 <br/>
Similarly, $M_3$ is 3.2 <br/>
So, the inter median/quartile range (IQR) = 3.2 - 2.6 = 0.6 <br/>
Now we can compute the limits for outliers. <br/>

Lower limit : $M_1 - 1.5\times IQR = 2.6 - 1.5 \times 0.6 = 1.7$ <br/>
Similarly, using second equation we can determine the upper limit, <br/>
Upper limit : $ 3.2 + 1.5 \times 0.6 = 4.1$
</div>

</body>
</html>

In [29]:
hide_me
answer642 = widgets.RadioButtons(options=['Select best one', '2.2', '3.8', '1.7', '2.1', 'None of the above'],
                             value = 'Select best one',  description='Choices:')

question642 = "6.4.2 What is the Potential outliers  of semester 1?"

def check642(g):
    IPython.display.clear_output(wait=False)
    display(question642, answer642)
    if answer642.value == '2.1':
        print("Correct Answer!")
    else:
        if answer642.value == 'Select best one':
            pass
        else:
            print("Wrong answer! Try again.")

IPython.display.clear_output(wait=False)
display(question642, answer642)
answer642.observe(check642, 'value')

6.4.2 What is the Potential outliers  of semester 1?


RadioButtons(description='Choices:', options=('Select best one', '2.2', '3.8', '1.7', '2.1', 'None of the above'), value='Select best one')

In [30]:
%%html
<html>
<head>
<script type="text/javascript">
<!--
function toggle(id) {
var e = document.getElementById(id);
if(e.style.display == 'none')
e.style.display = 'block';
else
e.style.display = 'none';
}
//-->
</script>
</head>

<body>
<div id="question642">
 <button id = "642"
onclick="toggle('answer642');">Solution</button> 
</div>
<div style="display:none" id="answer642">
In our previous question we determined the lower and upper limit of semester 2. Similarly if we calculate the limits for semester 1 
we find that the lower and upper limits are 2.2 and 3.8 respectively. So as per the definition of outliers, the numbers that are 
lower than the lower limit and higher than the upper limit should be the potential outliers.<br />

So from the given data of semester 1 we can find that 2.1 is less than the lower limit (2.2)and all other GPAs are in between lower
and upper limit.  <br/>Hence, 2.1 is potential outlier.
</div>

</body>
</html>

![alt text][bottom-banner]

[bottom-banner]: ./callysto-bottom-banner.jpg